In [ ]:
models = {'xgb':xgb_cl,
          'bag': bag_clf,
          'lgb': lgb_clf,
          'rf':rf_final,
          'AdaBoost': adaModel}
stacked_model = stack_models(*models.items(), cv_folds=3)
stacked_model.fit(X_train, y_train)
models['stacked'] = stacked_model

In [ ]:
#Creating a temporary dataframe so we can see how each of our models performed
temp = pd.DataFrame(y_test)

# The stacked models predictions, which should perform the best
temp['stacking_prediction'] = stacked_model.predict(X_test)

# Get each model in the stacked model to see how they individually perform
for m in stacked_model.named_estimators_:
        temp[m] = stacked_model.named_estimators_[m].predict(X_test)

In [ ]:
# See how each of our models correlate with our target
# In most instances of running the program the stacked predictions should outperform any singular model
print("Correlations with target column")
print(temp.corr()['isFraud'])

In [ ]:
from matplotlib import pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

# evaluate models using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=1)
	scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
	return scores

In [ ]:
# evaluate the models and store results
results, names = list(), list()
for name, m in models.items():
	scores = evaluate_model(m, X_test, y_test)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

In [ ]:
from utils.diagnostics import get_diagnostics
get_diagnostics(*models.items(), y_true=y_test, X=X_test)